In [ ]:
import networkx as nx
from cdlib import *
from cdlib import viz
from collections import OrderedDict
def read_com(f_c):
    #返回按照社团中节点数量排序的社团-节点字典
    com = {}
    coms = OrderedDict()
    with open(f_c, 'r', encoding='utf-8') as f:
        for n in f.readlines():
            n_c = n.strip('\n').split(':')
            n = n_c[0]
            # daikan
            c = n_c[1].split(' ')
            com[n] = c
    com = dict(sorted(com.items(), reverse=True, key=lambda k: len(k[1])))
    for i, c in enumerate(com.values()):
        coms[i] = c
    
    return coms
def turnto_community_graph(graph, partition, figsize=(8, 8), node_size=200, plot_overlaps=False, plot_labels=False):
    """
        Plot a algorithms-graph with node color coding for communities.

        :param graph: NetworkX/igraph graph
        :param partition: NodeClustering object
        :param figsize: the figure size; it is a pair of float, default (8, 8)
        :param node_size: int, default 200
        :param plot_overlaps: bool, default False. Flag to control if multiple algorithms memberships are plotted.
        :param plot_labels: bool, default False. Flag to control if node labels are plotted.

        Example:

        >>> from cdlib import algorithms, viz
        >>> import networkx as nx
        >>> g = nx.karate_club_graph()
        >>> coms = algorithms.louvain(g)
        >>> viz.plot_community_graph(g, coms)
        """

#     cms = partition.communities

    node_to_com = {}
    com_size={}
    for cid, com in partition.items():
        com_size[cid]=len(com)
        for node in com:
            if node not in node_to_com:
                node_to_com[node] = cid
            else:
                # duplicating overlapped node
                alias = "%s_%s" % (node, cid)
                node_to_com[alias] = cid
                edges = [(alias, y) for y in graph.neighbors(node)]
                graph.add_edges_from(edges)

    # handling partial coverage
    s = nx.subgraph(graph, node_to_com.keys())

    # algorithms graph construction
    c_graph = induced_graph(node_to_com, s)
    node_cms = [[node] for node in c_graph.nodes()]
    viz.plot_network_clusters(c_graph, NodeClustering(node_cms, None, ""), nx.spring_layout(c_graph), figsize=figsize,
                                 node_size=node_size, plot_overlaps=plot_overlaps, plot_labels=plot_labels)

    return c_graph,com_size

def induced_graph(partition, graph, weight="weight"):
    """Produce the graph that nodes are the communities

    there is a link of weight w between communities if the sum of the weights
    of the links between their elements is w

    Parameters
    ----------
    partition : dict
       a dictionary where keys are graph nodes and  values the part the node
       belongs to
    graph : networkx.Graph
        the initial graph
    weight : str, optional
        the key in graph to use as weight. Default to 'weight'


    Returns
    -------
    g : networkx.Graph
       a networkx graph where nodes are the parts

    Examples
    --------
    >>> n = 5
    >>> g = nx.complete_graph(2*n)
    >>> part = dict([])
    >>> for node in g.nodes() :
    >>>     part[node] = node % 2
    >>> ind = induced_graph(part, g)
    >>> goal = nx.Graph()
    >>> goal.add_weighted_edges_from([(0,1,n*n),(0,0,n*(n-1)/2), (1, 1, n*(n-1)/2)])  # NOQA
    >>> nx.is_isomorphic(int, goal)
    True
    """
    ret = nx.Graph()
    ret.add_nodes_from(partition.values())

    for node1, node2, datas in graph.edges(data=True):
        edge_weight = datas.get(weight, 1)
        com1 = partition[node1]
        com2 = partition[node2]
        w_prec = ret.get_edge_data(com1, com2, {weight: 0}).get(weight, 1)
        ret.add_edge(com1, com2, **{weight: w_prec + edge_weight})

    return ret
def add_node_weight(G,node_att):
    for k,v in node_att.items():
        G.nodes[k]['weight']=v
    return G
 #将划分得到的图转化为以社团为节点的图，主要改变了cdlib中的函数
g_list=['PRPER','PRAB','PRFLUIDS','PRI','PRSTPER','PRAPPLIED','PRX','PRSTAB','RMP',
       'PRE','PR','PRC','PRA','PRD']
fgs='F:/aps_analysis/graph_label/graph/'
fcs='F:/aps_analysis/graph_label/community/'
fo='F:/aps_analysis/graph_label/output_new/com_graph/'
font1 = {'family' : 'Times New Roman',
             'weight' : 'normal',
             'size'   : 10}
for g in g_list:
    fg=fgs+g+'.gexf'
    fc=fcs+g+'.txt'
    G=nx.read_gexf(fg)
    coms=read_com(fc)
#     partition=NodeClustering(list(coms.values()), None, "")
    c_g,com_size=turnto_community_graph(G,coms)
    p=nx.selfloop_edges(c_g)
    c_g.remove_edges_from(p)
    c_g=add_node_weight(c_g,com_size)
    nx.write_gexf(c_g,fo+g+'_com.gexf')
    plt.title(g,fontdict=font1,y=-0.1)
    plt.savefig(fo+g+'_com.png',dpi=300)
       

In [ ]:
#绘制社团节点标签
import json

fls='F:/aps_analysis/graph_label/output_new/com_label_order/'
fws='F:/aps_analysis/graph_label/output_new/com_word/'
fo='F:/aps_analysis/graph_label/output_new/add_com_label/'
# fo='F:/aps_analysis/graph_label/output_new/add_com_label_nomax/'
g_list=['PRPER','PRAB','PRFLUIDS','PRI','PRSTPER','PRAPPLIED','PRX','PRSTAB','RMP',
       'PRE','PR','PRC','PRA','PRD']
for g in g_list:
    fl=fls+g+'.json'
    fw=fws+g+'.json'
    with open(fl,'r',encoding='utf-8')as f:
        l=json.load(f)
    with open(fw,'r',encoding='utf-8')as f1:
        w=json.load(f1)
    new_l={}
    for k,v in l.items():
        d={}
        s=w[k]
        max_k1=sorted(v.items(),reverse=True,key= lambda k:k[1])[0][0]
        for k1,v1 in v.items():
            d[k1]=s.get(k1,0)+v1
#             if k1==max_k1:
#                 d[k1]=v1
#             else:   
#                 d[k1]=s.get(k1,0)+v1
        order_s=sorted(s.items(),reverse=True,key= lambda k:k[1])
#         print(order_s)
        i=0
        m=len(order_s)
        while len(d)<21:
            if order_s[i][0] in d.keys():
                pass
            else:
                d[order_s[i][0]]=order_s[i][1]
            i=i+1
            if i==m:
                break
        d=dict(sorted(d.items(),reverse=True,key= lambda k:k[1]))
        new_l[k]=d
    with open(fo+g+'.json','w',encoding='utf-8')as f2:
        json.dump(new_l,f2)


In [ ]:
#生成词云图
from PIL import Image
from PIL import ImageColor
import json
import numpy as np
from matplotlib import pyplot as plt
from wordcloud import WordCloud,STOPWORDS
def read_text(ft):
    with open(ft,'r',encoding='utf-8')as f:
        data=json.load(f)
    return data
def getpart(data,n_list):
    dt={}
    for n in n_list:
        dt[n]=data[n]
    return dt

def label_wordcloud(fileout,name,label,image_file):
    #com_label  {com:{label:fre}}
    
        # print(label)

        # read the mask image
        # taken from
        # http://www.stencilry.org/stencils/movies/alice%20in%20wonderland/255fk.jpg
        alice_mask = np.array(Image.open(image_file))
        stopwords = set(STOPWORDS)
        # stopwords.add("said")
        wc = WordCloud(background_color="white", max_words=15, 
#                        width=20,height=16,
#             mask=alice_mask,
#           max_font_size=36,
            stopwords=stopwords, contour_width=3, contour_color='white',
#           colormap='gist_heat'
            colormap='viridis'
#                        colormap='inferno'
                      )
        # generate word cloud
        wc.generate_from_frequencies(label)

        # store to file
#         wc.to_file(fileout+'{}_wordcloud.png'.format(com.split('.')[-1]))
        # show
        plt.imshow(wc, interpolation='bilinear')
        plt.axis("off")
#         plt.figure()
#         plt.imshow(alice_mask, cmap=plt.cm.gray, interpolation='bilinear')
#         plt.axis("off")
        plt.savefig(fileout+'{}_wordcloud.png'.format(name),dpi=500)
        plt.show()
prstab=["10.1103/PhysRevSTAB.13.092803","10.1103/PhysRevSTAB.7.074401","10.1103/PhysRevSTAB.17.121001",
        "10.1103/PhysRevSTAB.6.104402","10.1103/PhysRevSTAB.16.010705","10.1103/PhysRevSTAB.6.101301",
       "10.1103/PhysRevSTAB.5.084001","10.1103/PhysRevSTAB.13.080401","10.1103/PhysRevSTAB.18.091001",
       "10.1103/PhysRevSTAB.13.121004"]
rmp=["10.1103/RevModPhys.77.721","10.1103/RevModPhys.79.135", "10.1103/RevModPhys.25.390",
    "10.1103/RevModPhys.59.671","10.1103/RevModPhys.46.597"]
prx=["10.1103/PhysRevX.8.011040","10.1103/PhysRevX.7.021003","10.1103/PhysRevX.7.011014",
    "10.1103/PhysRevX.4.021017","10.1103/PhysRevX.7.011001"]
image_file='D:/Backup Plus/slpa-dataset/slpa_compare_input/logo.png'
# fo='F:/aps_analysis/graph_label/output_new/wordcloud/prstab/adjust/'
# fi='F:/aps_analysis/graph_label/output_new/add_com_label_nomax - adjust/PRSTAB.json'
fo='F:/aps_analysis/graph_label/output_new/prstab/'
fi='F:/aps_analysis/graph_label/output_new/add_com_label_nomax - adjust/PRSTAB.json'

data=read_text(fi)
dt=getpart(data,prstab)
for c,v in dt.items():
    name=c.split('.')[-1]
    label_wordcloud(fo,name,v,image_file)